# Práctica Big Data processing
En esta practica, utilizaremos dos datasets que contienen información de cada pais sobre varios indicadores de felicidad. 
  - **df** será el primer dataset, en el se incluyen datos hasta el año 2020.
  - **df2021**, el segundo satset en el que solamente se incluye datos del año 2021.

Con estos datos, se pretende poder responder las 6 cuestiones planteadas para la práctica.

## En primer lugar,
Vamos a instalar e importar todas las librerias que vamos a utilizar a continuación, para así poder tener un cuaderno mas limpio. 

In [1]:
#Instalamos Pyspark para poder utilizarlo en caso de no tenerlo

#pip install pyspark

#comprobamos la version de pyspark y de phyton
import pyspark
print(pyspark.__version__)


3.5.0


In [2]:
#Creamos una sesion de spark
from pyspark.sql import SparkSession

#Creamos un objeto de spark session
spark = SparkSession.builder.appName("keepcoding_rgp").getOrCreate()

from pyspark.sql import functions as F
from collections import Counter

A continuación, vamos a cargar nuestros datos base:

In [3]:
#Leemos un cvs
df = spark.read.option("header", "true").csv("C:/Users/Rocio/OneDrive - Universidad de Córdoba/.KEEPCODING/big data, IA, ML/7.big-data-processing/PROYECTO FINAL/Datasets/world-happiness-report.csv")
df2021 = spark.read.option("header", "true").csv("C:/Users/Rocio/OneDrive - Universidad de Córdoba/.KEEPCODING/big data, IA, ML/7.big-data-processing/PROYECTO FINAL/Datasets/world-happiness-report-2021.csv")

#Mostramos los primeros 10 registros
df.show(10)
df2021.show(10)

+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
|Country name|year|Life Ladder|Log GDP per capita|Social support|Healthy life expectancy at birth|Freedom to make life choices|Generosity|Perceptions of corruption|Positive affect|Negative affect|
+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
| Afghanistan|2008|      3.724|             7.370|         0.451|                          50.800|                       0.718|     0.168|                    0.882|          0.518|          0.258|
| Afghanistan|2009|      4.402|             7.540|         0.552|                          51.200|                       0.679|     0.190|                    0.850|          0.584|          0.237|
| Afghanistan|2

## Preguntas:


1. ¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Ladder score” mayor número más feliz es el país)


In [4]:

# Ordenamos la tabla con los valores de life ladder
max_life_ladder_country = df2021.orderBy(F.desc("Ladder score"))

print("Tabla ordenada por 'Ladder score':")
max_life_ladder_country.show(10)

#Seleccionamos el de mayor valor
max_country = max_life_ladder_country.select("country name", "Ladder score").first()
# Imprime el resultado
print(f"El pais con el mayor valor en 'Ladder score' es: {max_country['country name']}")


Tabla ordenada por 'Ladder score':
+------------+--------------------+------------+------------------------------+------------+------------+---------------------+--------------+-----------------------+----------------------------+----------+-------------------------+------------------------+--------------------------------+----------------------------+-------------------------------------+------------------------------------------+------------------------+---------------------------------------+-------------------+
|Country name|  Regional indicator|Ladder score|Standard error of ladder score|upperwhisker|lowerwhisker|Logged GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|Ladder score in Dystopia|Explained by: Log GDP per capita|Explained by: Social support|Explained by: Healthy life expectancy|Explained by: Freedom to make life choices|Explained by: Generosity|Explained by: Perceptions of corruption|Dystopia + res


2. ¿Cuál es el país más “feliz” del 2021 por continente según la data?


In [5]:

# Obtener la lista de continentes únicos
continentes = df2021.select("Regional indicator").distinct().collect()

# Crear una lista para almacenar los resultados
resultados = []

# Iterar sobre cada continente
for continente in continentes:
    # Filtrar el DataFrame para el continente actual
    df_cont = df2021.filter(df2021["Regional indicator"] == continente["Regional indicator"])
    
    # Obtener el país con el mayor "Ladder score" para el continente actual
    pais_mas_feliz = df_cont.orderBy(F.desc("Ladder score")).select("Regional indicator","country name", "Ladder score",).first()
    
    # Agregar a la lista de resultados
    resultados.append({
        "Continente": pais_mas_feliz["Regional indicator"],
        "País más feliz": pais_mas_feliz["country name"],
        "Ladder Score": pais_mas_feliz["Ladder score"]
    })

for resultado in resultados:
    print(resultado)

{'Continente': 'South Asia', 'País más feliz': 'Nepal', 'Ladder Score': '5.269'}
{'Continente': 'Middle East and North Africa', 'País más feliz': 'Israel', 'Ladder Score': '7.157'}
{'Continente': 'North America and ANZ', 'País más feliz': 'New Zealand', 'Ladder Score': '7.277'}
{'Continente': 'Sub-Saharan Africa', 'País más feliz': 'Mauritius', 'Ladder Score': '6.049'}
{'Continente': 'East Asia', 'País más feliz': 'Taiwan Province of China', 'Ladder Score': '6.584'}
{'Continente': 'Commonwealth of Independent States', 'País más feliz': 'Uzbekistan', 'Ladder Score': '6.179'}
{'Continente': 'Latin America and Caribbean', 'País más feliz': 'Costa Rica', 'Ladder Score': '7.069'}
{'Continente': 'Western Europe', 'País más feliz': 'Finland', 'Ladder Score': '7.842'}
{'Continente': 'Central and Eastern Europe', 'País más feliz': 'Czech Republic', 'Ladder Score': '6.965'}
{'Continente': 'Southeast Asia', 'País más feliz': 'Singapore', 'Ladder Score': '6.377'}



3. ¿Cuál es el país que más veces ocupó el primer lugar en todos los años?    
Se usará el dataset **df**

In [6]:
# Obtener la lista de los años que hay registros
years = df.select("year").distinct()

# Iterar sobre cada año
for year_row in years.collect():
    year = year_row["year"]
    
    # Filtrar el DataFrame para el año actual
    pais_mas_feliz = df.filter(df["year"] == year).orderBy(F.desc("Life Ladder")).select("year", "country name", "Life Ladder").first()
    
    # Imprimir el resultado
    print({
        "Año": pais_mas_feliz["year"],
        "País más feliz": pais_mas_feliz["country name"],
        "Life Ladder": pais_mas_feliz["Life Ladder"]
    })


{'Año': '2016', 'País más feliz': 'Finland', 'Life Ladder': '7.660'}
{'Año': '2012', 'País más feliz': 'Switzerland', 'Life Ladder': '7.776'}
{'Año': '2020', 'País más feliz': 'Finland', 'Life Ladder': '7.889'}
{'Año': '2019', 'País más feliz': 'Finland', 'Life Ladder': '7.780'}
{'Año': '2017', 'País más feliz': 'Finland', 'Life Ladder': '7.788'}
{'Año': '2014', 'País más feliz': 'Denmark', 'Life Ladder': '7.508'}
{'Año': '2013', 'País más feliz': 'Canada', 'Life Ladder': '7.594'}
{'Año': '2005', 'País más feliz': 'Denmark', 'Life Ladder': '8.019'}
{'Año': '2009', 'País más feliz': 'Denmark', 'Life Ladder': '7.683'}
{'Año': '2018', 'País más feliz': 'Finland', 'Life Ladder': '7.858'}
{'Año': '2006', 'País más feliz': 'Finland', 'Life Ladder': '7.672'}
{'Año': '2011', 'País más feliz': 'Denmark', 'Life Ladder': '7.788'}
{'Año': '2008', 'País más feliz': 'Denmark', 'Life Ladder': '7.971'}
{'Año': '2007', 'País más feliz': 'Denmark', 'Life Ladder': '7.834'}
{'Año': '2015', 'País más feliz

In [7]:
# Obtener la lista de continentes únicos
years = df.select("year").distinct().collect()

# Crear una lista para almacenar los resultados
resultados = []

# Iterar sobre cada continente
for year in years:
    # Filtrar el DataFrame para el continente actual
    df_cont = df.filter(df["year"] == year["year"])
    
    # Obtener el país con el mayor "Ladder score" para el continente actual
    pais_mas_feliz = df_cont.orderBy(F.desc("Life Ladder")).select("year", "country name", "Life Ladder").first()
    
    # Agregar a la lista de resultados
    resultados.append({
        "Año": pais_mas_feliz["year"],
        "País más feliz": pais_mas_feliz["country name"],
        "Life Ladder": pais_mas_feliz["Life Ladder"]
    })

# Imprimir la lista de resultados
for resultado in resultados:
    print(resultado)

#-------------------------

#Una vez que tenemos la lista con los paises n1 en cada año, contamos su frecuencia

# Extraer los nombres de los países
paises = [resultado["País más feliz"] for resultado in resultados]

# Contar la frecuencia de cada país
frecuencia_paises = Counter(paises)

# Obtener el país más común
pais_mas_comun = frecuencia_paises.most_common(1)[0][0]

# Imprimir el país más común
print(f"El país más común en la lista de resultados es: {pais_mas_comun}.")

{'Año': '2016', 'País más feliz': 'Finland', 'Life Ladder': '7.660'}
{'Año': '2012', 'País más feliz': 'Switzerland', 'Life Ladder': '7.776'}
{'Año': '2020', 'País más feliz': 'Finland', 'Life Ladder': '7.889'}
{'Año': '2019', 'País más feliz': 'Finland', 'Life Ladder': '7.780'}
{'Año': '2017', 'País más feliz': 'Finland', 'Life Ladder': '7.788'}
{'Año': '2014', 'País más feliz': 'Denmark', 'Life Ladder': '7.508'}
{'Año': '2013', 'País más feliz': 'Canada', 'Life Ladder': '7.594'}
{'Año': '2005', 'País más feliz': 'Denmark', 'Life Ladder': '8.019'}
{'Año': '2009', 'País más feliz': 'Denmark', 'Life Ladder': '7.683'}
{'Año': '2018', 'País más feliz': 'Finland', 'Life Ladder': '7.858'}
{'Año': '2006', 'País más feliz': 'Finland', 'Life Ladder': '7.672'}
{'Año': '2011', 'País más feliz': 'Denmark', 'Life Ladder': '7.788'}
{'Año': '2008', 'País más feliz': 'Denmark', 'Life Ladder': '7.971'}
{'Año': '2007', 'País más feliz': 'Denmark', 'Life Ladder': '7.834'}
{'Año': '2015', 'País más feliz


4. ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?

In [8]:

filter_country = df.filter(df.year == 2020)

# Ordenamos la tabla con los valores de Log GDP per capita
max_GDP_country = filter_country.orderBy(F.desc("Log GDP per capita"))


print("Tabla ordenada por 'Log GDP per capita':")
max_GDP_country.show(10)

#Seleccionamos el de mayor valor
max_country = max_GDP_country.select("country name", "Log GDP per capita","Life Ladder" ).first()
# Imprime el resultado
print(f"En 2020, el pais con mayor valor en 'Log GDP per capita' es {max_country['country name']} con una felicidad de {max_country['Life Ladder']}.")

Tabla ordenada por 'Log GDP per capita':
+------------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
|      Country name|year|Life Ladder|Log GDP per capita|Social support|Healthy life expectancy at birth|Freedom to make life choices|Generosity|Perceptions of corruption|Positive affect|Negative affect|
+------------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
|          Bulgaria|2020|      5.598|             9.991|         0.916|                          67.200|                       0.818|    -0.004|                    0.901|          0.706|          0.221|
|         Mauritius|2020|      6.015|             9.972|         0.893|                          67.000|                       0.843|    -0.037|   

5. ¿En que porcentaje a variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó o disminuyó?


In [9]:
df_2020 = df.filter(df.year == 2020)


# Calcular la suma total de Log GDP per capita para el año 2020
total_log_gdp = df_2020.agg(F.sum("Log GDP per capita").alias("total_log_gdp")).collect()[0]["total_log_gdp"]

# Agregar una columna con el porcentaje total a cada fila
df_percent = df_2020.withColumn(
    "log_GDP_percent", 
    F.col("Log GDP per capita") / total_log_gdp * 100
)

# Mostrar el resultado
df_percent.show(10)

# Calcular el promedio de la columna log_GDP_percent
promedio_log_gdp_percent = df_percent.agg(F.avg("log_GDP_percent").alias("promedio_log_gdp_percent")).collect()[0]["promedio_log_gdp_percent"]

# Mostrar el resultado
print(f"El promedio de log_GDP_percent de 2020 es: {promedio_log_gdp_percent:.3f}")


+--------------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+------------------+
|        Country name|year|Life Ladder|Log GDP per capita|Social support|Healthy life expectancy at birth|Freedom to make life choices|Generosity|Perceptions of corruption|Positive affect|Negative affect|   log_GDP_percent|
+--------------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+------------------+
|             Albania|2020|      5.365|             9.497|         0.710|                          69.300|                       0.754|     0.007|                    0.891|          0.679|          0.265|1.1067255397573992|
|           Argentina|2020|      5.901|             9.850|         0.897|                          69.20

In [10]:

# Calcular la suma total de Log GDP per capita para el año 2021
total_log_gdp_2021 = df2021.agg(F.sum("Logged GDP per capita").alias("total_log_gdp_2021")).collect()[0]["total_log_gdp_2021"]

# Agregar una columna con el porcentaje total a cada fila
df_percent_2021 = df2021.withColumn(
    "Logged GDP per capita", 
    F.col("Logged GDP per capita") / total_log_gdp_2021 * 100
)

# Mostrar el resultado
df_percent_2021.show(10)

# Calcular el promedio de la columna log_GDP_percent
promedio_log_gdp_percent_2021 = df_percent_2021.agg(F.avg("Logged GDP per capita").alias("promedio_log_gdp_percent_2021")).collect()[0]["promedio_log_gdp_percent_2021"]

# Mostrar el resultado
print(f"El promedio de log_GDP_percent de 2021 es: {promedio_log_gdp_percent_2021:.3f}")


+------------+--------------------+------------+------------------------------+------------+------------+---------------------+--------------+-----------------------+----------------------------+----------+-------------------------+------------------------+--------------------------------+----------------------------+-------------------------------------+------------------------------------------+------------------------+---------------------------------------+-------------------+
|Country name|  Regional indicator|Ladder score|Standard error of ladder score|upperwhisker|lowerwhisker|Logged GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|Ladder score in Dystopia|Explained by: Log GDP per capita|Explained by: Social support|Explained by: Healthy life expectancy|Explained by: Freedom to make life choices|Explained by: Generosity|Explained by: Perceptions of corruption|Dystopia + residual|
+------------+--------------

En el año 2020 se tuvo un promedio de GDP de 1.136 y en 2021 de 0.671, por lo que se ha disminuido.  

6. ¿Cuál es el país con mayor expectativa de vida (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia en ese indicador en el 2019?

In [11]:

# Ordenamos la tabla con los valores de Healthy life expectancy
max_healthy_countries = df2021.orderBy(F.desc("Healthy life expectancy"))

print("Tabla ordenada por 'Healthy life expectancy':")
max_healthy_countries.show(10)

#Seleccionamos el de mayor valor
max_healthy_country = max_healthy_countries.select("country name", "Healthy life expectancy").first()
# Imprime el resultado
print(f"El pais con el mayor valor en 'Healthy life expectancy' es: {max_country['country name']}")


Tabla ordenada por 'Healthy life expectancy':


+--------------------+--------------------+------------+------------------------------+------------+------------+---------------------+--------------+-----------------------+----------------------------+----------+-------------------------+------------------------+--------------------------------+----------------------------+-------------------------------------+------------------------------------------+------------------------+---------------------------------------+-------------------+
|        Country name|  Regional indicator|Ladder score|Standard error of ladder score|upperwhisker|lowerwhisker|Logged GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|Ladder score in Dystopia|Explained by: Log GDP per capita|Explained by: Social support|Explained by: Healthy life expectancy|Explained by: Freedom to make life choices|Explained by: Generosity|Explained by: Perceptions of corruption|Dystopia + residual|
+-----------

In [18]:
bulgaria_2019 = df.filter((df.year == 2019) & (df["Country name"] == 'Bulgaria'))

# Imprime el DataFrame resultante
bulgaria_2019.show()

# Accede al valor de la columna 'Healthy life expectancy'
healthy_life = bulgaria_2019.select('Healthy life expectancy at birth').first()

# Imprime el resultado
print(f"Bulgaria en el año 2019 tenía una 'Healthy life expectancy' de {healthy_life['Healthy life expectancy at birth']}.")

+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
|Country name|year|Life Ladder|Log GDP per capita|Social support|Healthy life expectancy at birth|Freedom to make life choices|Generosity|Perceptions of corruption|Positive affect|Negative affect|
+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+
|    Bulgaria|2019|      5.108|            10.047|         0.948|                          67.000|                       0.822|    -0.109|                    0.943|          0.663|          0.200|
+------------+----+-----------+------------------+--------------+--------------------------------+----------------------------+----------+-------------------------+---------------+---------------+

Bulgaria en el